<a href="https://colab.research.google.com/github/Duyuyuy/Airbnb_Analysis_Project/blob/main/Data_Modeling%2BAssociation_Rules.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Project - Los Angeles Airbnb data analysis**

Trần Bảo Duy - ITDSIU20061


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!pip install fuzzywuzzy
!pip install pandas_profiling==3.2.0
!pip install folium
!pip install markupsafe==2.0.1

In [ ]:
import os
import re
import random
import numpy as np
import pandas as pd
import folium as fo
import seaborn as sns
import pandas_profiling
import plotly.express as px
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.patches as mpatches
%matplotlib inline

from scipy.stats import linregress
from sklearn.metrics import r2_score
from folium.plugins import BeautifyIcon
from statsmodels.formula.api import logit
from fuzzywuzzy import process
from fuzzywuzzy import fuzz

In [ ]:
os.chdir('/content/gdrive/My Drive/data')

##Modeling


Set up spark environment

In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-3.1.1-bin-hadoop3.2"
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Get:8 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,015 kB]
Hit:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:12 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,539 kB]
Hit:13 http://ppa.lau

In [ ]:
from pyspark import SparkConf, SparkContext

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars xgboost4j-spark.jar,xgboost4j.jar pyspark-shell'



In [ ]:
spark = SparkSession\
        .builder\
        .appName("spark_tree_plotting")\
        .config("spark.executor.cores", 3) \
        .config("spark.executor.instances", 5) \
        .master("local[4]")\
        .getOrCreate()
# spark.sparkContext.addPyFile("spark-tree-plotting-dabddc53bf7b5de1e04c6fd40bf9f1c70cc13a98.zip")




In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, VectorSlicer
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.regression import LinearRegression



**Predicting price of accomodation that have high review score.**

###Preprocessing before modeling

In [ ]:
filtered_data = data.query("review_scores_rating > 4 & review_scores_cleanliness > 4 & review_scores_accuracy > 4 & satisfied==1")
filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26648 entries, 0 to 34269
Data columns (total 27 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           26648 non-null  int64  
 1   host_id                      26648 non-null  int64  
 2   host_identity_verified       26648 non-null  int64  
 3   neighbourhood_cleansed       26648 non-null  object 
 4   latitude                     26648 non-null  float64
 5   longitude                    26648 non-null  float64
 6   property_type                26648 non-null  object 
 7   room_type                    26648 non-null  object 
 8   accommodates                 26648 non-null  int64  
 9   bedrooms                     26648 non-null  float64
 10  amenities                    26648 non-null  object 
 11  price                        26648 non-null  float64
 12  minimum_nights               26648 non-null  int64  
 13  maximum_nights  

In [ ]:
df=spark.createDataFrame(filtered_data.drop(columns=['id','host_id','amenities','review_scores_rating','review_scores_accuracy','review_scores_cleanliness','review_scores_checkin','review_scores_communication','review_scores_location','review_scores_value','satisfied','latitude','longitude']))
df.show(5)

+----------------------+----------------------+-------------+---------------+------------+--------+-----+--------------+--------------+----------------+-----------------+----------------+-----------+---------+
|host_identity_verified|neighbourhood_cleansed|property_type|      room_type|accommodates|bedrooms|price|minimum_nights|maximum_nights|availability_365|number_of_reviews|instant_bookable|num_of_bath|bath_type|
+----------------------+----------------------+-------------+---------------+------------+--------+-----+--------------+--------------+----------------+-----------------+----------------+-----------+---------+
|                     1|               Del Rey|         home|   Private room|           1|     1.0| 65.0|            30|          1125|             365|              226|               0|        1.0|        0|
|                     1|                Venice|  rental unit|Entire home/apt|           2|     1.0|205.0|            31|           330|             339|        

In [ ]:
categorical_columns = ['neighbourhood_cleansed', 'property_type','room_type']
others= [ele for ele in df.columns if ele not in categorical_columns]
others.remove('price')

In [ ]:
stringindexer_stages = [StringIndexer(inputCol=c, outputCol='strindexed_' + c) for c in categorical_columns]


In [ ]:
onehotencoder_stages = [OneHotEncoder(inputCol='strindexed_' + c, outputCol='onehot_' + c) for c in categorical_columns]

In [ ]:
feature_columns = ['onehot_' + c for c in categorical_columns]

feature_columns.extend(others)
feature_columns
vectorassembler_stage = VectorAssembler(inputCols=feature_columns, outputCol='features') 

In [ ]:
feature_columns

['onehot_neighbourhood_cleansed',
 'onehot_property_type',
 'onehot_room_type',
 'host_identity_verified',
 'accommodates',
 'bedrooms',
 'minimum_nights',
 'maximum_nights',
 'availability_365',
 'number_of_reviews',
 'instant_bookable',
 'num_of_bath',
 'bath_type']

In [ ]:
all_stages = stringindexer_stages + onehotencoder_stages + [vectorassembler_stage]
pipeline = Pipeline(stages=all_stages)

In [ ]:
pipeline_model = pipeline.fit(df)

In [ ]:
final_columns = ['features'] + ['price']
dfc = pipeline_model.transform(df).\
            select(final_columns)


In [ ]:
dfc.show(5)

+--------------------+-----+
|            features|price|
+--------------------+-----+
|(291,[54,263,279,...| 65.0|
|(291,[1,264,278,2...|205.0|
|(291,[2,271,279,2...| 80.0|
|(291,[36,264,279,...| 62.0|
|(291,[36,264,279,...| 75.0|
+--------------------+-----+
only showing top 5 rows



In [ ]:
training, test = dfc.randomSplit([0.8, 0.2], seed=1234)

###Random Forest Regressor

In [ ]:
rf = RandomForestRegressor(featuresCol='features', labelCol='price', numTrees=26, maxDepth=13, maxBins=7, checkpointInterval=12)  

In [ ]:
{param[0].name: param[1] for param in rf.extractParamMap().items()}

{'seed': -7008999573985494056,
 'maxDepth': 13,
 'maxBins': 7,
 'minInstancesPerNode': 1,
 'minInfoGain': 0.0,
 'maxMemoryInMB': 256,
 'cacheNodeIds': False,
 'checkpointInterval': 12,
 'impurity': 'variance',
 'subsamplingRate': 1.0,
 'numTrees': 26,
 'featureSubsetStrategy': 'auto',
 'leafCol': '',
 'minWeightFractionPerNode': 0.0,
 'bootstrap': True,
 'predictionCol': 'prediction',
 'featuresCol': 'features',
 'labelCol': 'price'}

In [ ]:
  # paramGrid = ParamGridBuilder() \
  #   .addGrid(rf.checkpointInterval, [int(x) for x in np.linspace(start = 10, stop = 14, num = 3)]) \
  #   .addGrid(rf.maxDepth, [int(x) for x in np.linspace(start = 10, stop = 15, num = 3)]) \
  #   .addGrid(rf.maxBins, [int(x) for x in np.linspace(start = 5, stop = 8, num = 3)]) \
  #   .addGrid(rf.numTrees, [int(x) for x in np.linspace(start = 20, stop = 27, num = 3)]) \
  #   .build()
  

In [ ]:
# tvs = TrainValidationSplit(estimator=rf,
#                            estimatorParamMaps=paramGrid,
#                            evaluator=rfevaluator,
#                            # 80% of the data will be used for training, 20% for validation.
#                            trainRatio=0.8,
#                            parallelism=8)

In [ ]:
#rf after tunning
rf = RandomForestRegressor(featuresCol='features', labelCol='price', numTrees=26, maxDepth=13, maxBins=7, checkpointInterval=12)  

In [ ]:
# model1 = tvs.fit(training)
model1=rf.fit(training)

In [ ]:
pred1=model1.transform(test)
pred_train1=model1.transform(training)

In [ ]:
rfevaluator = RegressionEvaluator(predictionCol="prediction", labelCol="price", metricName="rmse")

###Linear Regression

In [ ]:
lr = LinearRegression(featuresCol='features', labelCol='price', maxIter=12, elasticNetParam=1)
paramGrid2 = ParamGridBuilder()\
    .addGrid(lr.regParam, [0.1, 0.3,0.035]) \
    .addGrid(lr.fitIntercept, [False, True])\
    .addGrid(lr.epsilon, [1.4, 1.6, 1.7])\
    .addGrid(lr.aggregationDepth, [2, 4])\
    .build()
# .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])\

In [ ]:
{param[0].name: param[1] for param in lr.extractParamMap().items()}

{'maxBlockSizeInMB': 0.0,
 'aggregationDepth': 2,
 'solver': 'auto',
 'standardization': True,
 'fitIntercept': True,
 'elasticNetParam': 1.0,
 'predictionCol': 'prediction',
 'featuresCol': 'features',
 'labelCol': 'price',
 'maxIter': 12,
 'regParam': 0.0,
 'tol': 1e-06,
 'loss': 'squaredError',
 'epsilon': 1.35}

In [ ]:
tvs2 = TrainValidationSplit(estimator=lr,
                           estimatorParamMaps=paramGrid2,
                           evaluator=rfevaluator,
                           # 80% of the data will be used for training, 20% for validation.
                           trainRatio=0.8,
                           parallelism=5)

In [ ]:
model2 = tvs2.fit(training)


In [ ]:
pred2=model2.transform(test)
pred_train2=model2.transform(training)

###Assessing

Random Forest Regressor

In [ ]:
RMSE_train1 = rfevaluator.evaluate(pred_train1)
RMSE1 = rfevaluator.evaluate(pred1)
print(RMSE_train1,RMSE1)

117.87984353698418 148.75731962281043


Linear Regression

In [ ]:
RMSE_train2 = rfevaluator.evaluate(pred_train2)
RMSE2 = rfevaluator.evaluate(pred2)
print(RMSE_train2,RMSE2)

162.14901426830903 165.61972681977622


Feature importance

In [ ]:
def ExtractFeatureImp(featureImp, dataset, featuresCol):
    list_extract = []
    for i in dataset.schema[featuresCol].metadata["ml_attr"]["attrs"]:
        list_extract = list_extract + dataset.schema[featuresCol].metadata["ml_attr"]["attrs"][i]
    varlist = pd.DataFrame(list_extract)
    varlist['score'] = varlist['idx'].apply(lambda x: featureImp[x])
    return(varlist.sort_values('score', ascending = False))

In [ ]:
ExtractFeatureImp(model1.featureImportances, pred_train1, "features").head(10)


,idx,name,score
2,283,bedrooms,0.284401
8,289,num_of_bath,0.231200
1,282,accommodates,0.078536
279,269,onehot_property_type_villa,0.053941
34,24,onehot_neighbourhood_cleansed_Malibu,0.042663
288,278,onehot_room_type_Entire home/apt,0.027633
5,286,availability_365,0.027160
3,284,minimum_nights,0.025991
25,15,onehot_neighbourhood_cleansed_Hollywood Hills ...,0.023396
6,287,number_of_reviews,0.021867


##**Association rule with amenities**



In [ ]:
amenities = data['amenities']

In [ ]:
# define functions

def createArray(arr, gr):
  a = [0 for _ in range(len(gr.keys()))]
  l = list(gr.keys())

  for i in range(len(l)):
    a[i] = has(arr, gr[l[i]])
  
  return a

def has(arr, key):
  for k in key:
    k = k.lower()

    for i in arr:
      i = i.lower()
      if k in i:
        return 1
  return 0

In [ ]:
# Split strings of amenities into arrays of strings of amenities

arr = []

for i in amenities:
  l = re.sub('[^a-zA-Z0-9, \n\.]', '', i[1:-1]).split(', ')
  arr.append(l)

In [ ]:
len(arr)

34270

In [ ]:
# Define columns of services

gr = {'tv' : ['tv', 'hdtv'],
      'wifi' : ['wifi', 'ethernet'],
      'sound_system' : ['sound system'],
      'pool' : ['pool'],
      'gym' : ['gym'],
      'boardgame' : ['game'],
      'ping_pong' : ['ping pong'],
      'book' : ['book'],
      'piano' : ['Piano'],
      'bike_renting' : ['Bikes', 'bike', 'bicycle'],
      'kayak_renting' : ['Kayak'],
      'EV_charger' : ['EV charger'],
      'luggage_carrier' : ['luggage'],
      'streaming_service' : ['Netflix', 'HBO Max', 'Roku', 'Amazon Prime Video', 'Chromecast'],
      'cable' : ['premium cable', 'standard cable'],
      'record/DVD_player' : ['Record player', 'DVD player'],
      'baby_facilities' : ['crib', 'baby'],
      'personal_workplace' : ['workplace'],
      'parking_lot' : ['parking'],
      'elevator' : ['elevator'],
      'smoke_alarm' : ['smoke alarm'],
      'CO_alarm' : ['carbon monoxide alarm'],
      'fire_extinguisher' : ['extinguisher'],
      'first_aid_kit' : ['first aid'],
      'security' : ['security', 'guard', 'staff'],
      'private_entrance' : ['private entrance'],
      'private_patio/balcony' : ['Private patio or balcony'],
      'private_garden/backyard' : ['Private fenced garden or backyard', 'Private garden or backyard'],
      'outdoor_dining' : ['outdoor dining area'],
      'breakfast' : ['breakfast'],
      'long_stays' : ['Long term stays'],
      'air_conditioning' : ['conditioning'],
      'fan' : ['fan', 'fans'],
      'mosquito_net' : ['Mosquito net'],
      'wardrobe' : ['wardrobe', 'dresser'],
      'changing_table' : ['Changing table'],
      'has_kitchen' : ['kitchen'],
      'oven/stove/microwave' : ['oven', 'stove', 'microwave', 'Baking sheet'],
      'refridgerator/freezer' : ['refridgerator', 'freezer'],
      'coffee machine' : ['coffee', 'Nespresso'],
      'kettle' : ['kettle', 'hot water'],
      'rice cooker' : ['rice'],
      'toaster' : ['bread', 'Toaster'],
      'dishwasher' : ['diswasher'],
      'kitchen_wares' : ['dishes', 'dinnerware'],
      'BBQ_utilities' : ['Barbecue utensils', 'Fire pit', 'BBQ grill'],
      'wine' : ['wine'],
      'hanger' : ['hangers'],
      'hair_dryer' : ['hair dryer'],
      'bidet' : ['Bidet'],
      'shampoo/soap' : ['shampoo', 'soap'],
      'conditioner/body_wash' : ['conditioner', 'body wash'],
      'bathtub' : ['bathtub', 'tub'],
      'sauna' : ['sauna'],
      'ironing' : ['iron'],
      'clothes_dryer' : ['dryer'],
      'washer' : ['washer'],
      'closet' : ['closet'],
      'heater/fireplace' : ['heating', 'fireplace'],
      'others' : ['pillow', 'chair'],}

df = pd.DataFrame(columns = gr.keys())

In [ ]:
# Process the amenities from 0 to 9999

for i in range(0, 10000):
  df.loc[len(df)] = createArray(arr[i], gr)

In [ ]:
# Process the amenities from 10000 to 19999

for i in range(10000, 20000):
  df.loc[len(df)] = createArray(arr[i], gr)

In [ ]:
# Process the amenities from 20000 to 29999

for i in range(20000, 30000):
  df.loc[len(df)] = createArray(arr[i], gr)

In [ ]:
# Process the amenities from 30000 to end

for i in range(30000, len(arr)):
  df.loc[len(df)] = createArray(arr[i], gr)

In [ ]:
len(arr)

34270

In [ ]:
# add id column

df['id'] = data['id']

In [ ]:
# add satisfied column

df = df.join(data['satisfied']).astype({'satisfied' : int})

In [ ]:
# Create table amenities:

arr = []
for i in df.columns:
  arr.append(i)

amenities = pd.DataFrame({'id' : [], 'satisfied' : [], 'has' : []})

for i in range(len(df)):
  a = ''
  for j in arr:
    if df[j].loc[i] == 1:
      a = a + j + '|'
  amenities.loc[len(amenities)] = [str(df['id'].loc[i]), df['satisfied'].loc[i], a[:-1]]

In [ ]:
amenities = amenities.astype({'satisfied' : int})

Create a table based on *amenities* and *satisfied* column to show the services that satisfy customers:

In [ ]:
%pip install mlxtend --upgrade
from mlxtend.preprocessing import TransactionEncoder 
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import fpgrowth


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.5 MB/s eta 0:00:00
  Attempting uninstall: mlxtend
    Found existing installation: mlxtend 0.14.0
    Uninstalling mlxtend-0.14.0:
      Successfully uninstalled mlxtend-0.14.0


In [ ]:
array = []
for i in df[df['satisfied'] == 1].drop(columns = ['id', 'satisfied']).sum().sort_values(ascending = False).to_dict().values():
    array.append(round(i / len(df), 3))

f = {'Service' : list(df[df['satisfied'] == 1].drop(columns=['id', 'satisfied']).sum().sort_values(ascending = False).to_dict().keys()),
     'Count' : list(df[df['satisfied'] == 1].drop(columns=['id', 'satisfied']).sum().sort_values(ascending = False).to_dict().values()),
     'Percent' : array}
mode = pd.DataFrame.from_dict(f)

In [ ]:
mode[mode['Percent'] > 0.5]

,Service,Count,Percent
0,wifi,26720,0.780
1,smoke_alarm,25955,0.757
2,clothes_dryer,24795,0.724
3,parking_lot,24769,0.723
4,long_stays,24393,0.712
5,heater/fireplace,24305,0.709
6,has_kitchen,24301,0.709
7,tv,24006,0.700
8,CO_alarm,23679,0.691
9,hanger,22507,0.657


In [ ]:
h=df[df['satisfied']==1]
h.drop(columns=['satisfied'], inplace= True)



/usr/local/lib/python3.9/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [ ]:
h.set_index('id',inplace=True)
d=h[h==1].stack().reset_index().drop(0,1)
c=d.values.tolist()

<ipython-input-123-2fd5fc24577b>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  d=h[h==1].stack().reset_index().drop(0,1)


In [ ]:
# h=h.drop(columns=['id'])
# h

In [ ]:
a=list(set(d['id'].tolist()))
len(a)
a

[25034755,
 52428810,
 35258379,
 25034768,
 747219174767329298,
 13762581,
 28180503,
 50987045,
 14549031,
 552336028815065133,
 48365618,
 39059513,
 47317050,
 650432920476450874,
 51904569,
 562300456194408513,
 620444389462507587,
 718494618331906127,
 735780593043505239,
 770251091097813081,
 39845981,
 733350524022161501,
 748017353066676320,
 48365669,
 131174,
 748085550193180777,
 653839011088564331,
 40894582,
 731498185671835790,
 40239251,
 30802068,
 27787414,
 31195287,
 12714144,
 584611549318873249,
 52691106,
 45220003,
 739150654853873830,
 29884585,
 3932334,
 6029497,
 44433596,
 25952458,
 716226650962788556,
 617541805709590732,
 53215440,
 45088983,
 613078808511906008,
 35389657,
 609022117821677786,
 46530780,
 53346526,
 8519906,
 51904742,
 558205075396755687,
 11534569,
 4325638,
 39059721,
 732916169283993867,
 52953358,
 262430,
 46661918,
 31981857,
 53215522,
 24379701,
 551505851666137403,
 50856254,
 31719747,
 53346637,
 46006606,
 48890194,
 622016

In [ ]:
k=[]
for x in a:
  v=[]
  for y in c:
    if x==y[0]:
      v.append(y[1])
  k.append(v)

In [ ]:
c

[[16985679, 'wifi'],
 [16985679, 'luggage_carrier'],
 [16985679, 'parking_lot'],
 [16985679, 'smoke_alarm'],
 [16985679, 'CO_alarm'],
 [16985679, 'fire_extinguisher'],
 [16985679, 'first_aid_kit'],
 [16985679, 'private_entrance'],
 [16985679, 'outdoor_dining'],
 [16985679, 'long_stays'],
 [16985679, 'has_kitchen'],
 [16985679, 'oven/stove/microwave'],
 [16985679, 'coffee machine'],
 [16985679, 'kettle'],
 [16985679, 'kitchen_wares'],
 [16985679, 'BBQ_utilities'],
 [16985679, 'hanger'],
 [16985679, 'hair_dryer'],
 [16985679, 'shampoo/soap'],
 [16985679, 'ironing'],
 [16985679, 'clothes_dryer'],
 [16985679, 'washer'],
 [16985679, 'heater/fireplace'],
 [16985679, 'others'],
 [49093882, 'tv'],
 [49093882, 'wifi'],
 [49093882, 'bike_renting'],
 [49093882, 'parking_lot'],
 [49093882, 'smoke_alarm'],
 [49093882, 'CO_alarm'],
 [49093882, 'fire_extinguisher'],
 [49093882, 'first_aid_kit'],
 [49093882, 'outdoor_dining'],
 [49093882, 'long_stays'],
 [49093882, 'air_conditioning'],
 [49093882, 'fa

In [ ]:
k

[['tv',
  'wifi',
  'pool',
  'book',
  'luggage_carrier',
  'parking_lot',
  'smoke_alarm',
  'CO_alarm',
  'fire_extinguisher',
  'first_aid_kit',
  'security',
  'private_entrance',
  'private_patio/balcony',
  'outdoor_dining',
  'long_stays',
  'has_kitchen',
  'oven/stove/microwave',
  'coffee machine',
  'kettle',
  'kitchen_wares',
  'BBQ_utilities',
  'hanger',
  'hair_dryer',
  'shampoo/soap',
  'bathtub',
  'ironing',
  'clothes_dryer',
  'washer',
  'heater/fireplace'],
 ['tv',
  'wifi',
  'luggage_carrier',
  'parking_lot',
  'smoke_alarm',
  'CO_alarm',
  'first_aid_kit',
  'private_entrance',
  'outdoor_dining',
  'long_stays',
  'air_conditioning',
  'fan',
  'has_kitchen',
  'oven/stove/microwave',
  'coffee machine',
  'kettle',
  'toaster',
  'kitchen_wares',
  'BBQ_utilities',
  'hanger',
  'hair_dryer',
  'shampoo/soap',
  'conditioner/body_wash',
  'bathtub',
  'ironing',
  'clothes_dryer',
  'heater/fireplace',
  'others'],
 ['tv',
  'wifi',
  'book',
  'luggage_

In [ ]:
frequent_itemsets_fp=fpgrowth(h, min_support=0.6, use_colnames=True)
frequent_itemsets_fp

/usr/local/lib/python3.9/dist-packages/mlxtend/frequent_patterns/fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.990290,(wifi)
1,0.961938,(smoke_alarm)
2,0.918946,(clothes_dryer)
3,0.917982,(parking_lot)
4,0.904047,(long_stays)
...,...,...
7083,0.615151,"(wifi, shampoo/soap, air_conditioning)"
7084,0.602920,"(shampoo/soap, clothes_dryer, air_conditioning)"
7085,0.601772,"(smoke_alarm, shampoo/soap, air_conditioning)"
7086,0.601067,"(wifi, shampoo/soap, clothes_dryer, air_condit..."


In [ ]:
rules_fp = association_rules(frequent_itemsets_fp, metric="confidence", min_threshold=0.8)
rules_fp

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(wifi),(smoke_alarm),0.990290,0.961938,0.955748,0.965120,1.003308,0.003151,1.091231
1,(smoke_alarm),(wifi),0.961938,0.990290,0.955748,0.993566,1.003308,0.003151,1.509148
2,(wifi),(clothes_dryer),0.990290,0.918946,0.913461,0.922418,1.003778,0.003438,1.044749
3,(clothes_dryer),(wifi),0.918946,0.990290,0.913461,0.994031,1.003778,0.003438,1.626783
4,(smoke_alarm),(clothes_dryer),0.961938,0.918946,0.890816,0.926064,1.007746,0.006847,1.096278
...,...,...,...,...,...,...,...,...,...
160706,"(wifi, clothes_dryer, air_conditioning)",(shampoo/soap),0.709881,0.795197,0.601067,0.846716,1.064788,0.036573,1.336104
160707,"(shampoo/soap, clothes_dryer, air_conditioning)",(wifi),0.602920,0.990290,0.601067,0.996926,1.006702,0.004001,3.159303
160708,"(shampoo/soap, air_conditioning)","(wifi, clothes_dryer)",0.617337,0.913461,0.601067,0.973645,1.065886,0.037154,3.283560
160709,"(clothes_dryer, air_conditioning)","(wifi, shampoo/soap)",0.713698,0.791787,0.601067,0.842187,1.063654,0.035970,1.319367
